In [36]:
import helper_functions
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pickle

# LSTM
from keras.models import Sequential  
from keras.layers import Dense, Conv1D, Activation 
from keras.layers import InputLayer, Dropout, LSTM, BatchNormalization

import helper_functions

# Load Data 


In [37]:
# I am not using the helper function as my path is different (Alex)
data_path = 'data/processed/data'
with open(data_path, 'rb') as f:
    data = pickle.Unpickler(f).load()

In [38]:
# Shuffle the data
dataToShuffle = []

for log, mfcc, label in zip(data['log-mel'],data['mfccs'],data['classes']):
    dataToShuffle.append([log,mfcc,label])

ShuffledData = np.array(dataToShuffle)
np.random.shuffle(ShuffledData)

<ipython-input-38-9fd0e306c2b1>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ShuffledData = np.array(dataToShuffle)


In [39]:
# Separate the data into training and test datasets
Train = ShuffledData[:45305]
Test = ShuffledData[45305:]

log_train = []
mfcc_train = []
y_train = []

log_test = []
mfcc_test = []
y_test = []

for log, mfcc, label in Train:
    log_train.append(log)
    mfcc_train.append(mfcc)
    y_train.append(label)
    
for log, mfcc, label in Test:
    log_test.append(log)
    mfcc_test.append(mfcc)
    y_test.append(label)


# Create the model 


In [40]:
InputShape = log_test[0].shape
output_dim = y_test[0].shape[0]

dropout = 0.2


# Creation of the Convolutional LSTM
CLSTM = Sequential()
CLSTM.add(InputLayer(input_shape = InputShape))
CLSTM.add(Conv1D(filters=256, kernel_size=10, strides=4, name='conv_1'))
CLSTM.add(Activation('relu'))
CLSTM.add(BatchNormalization())
CLSTM.add(Dropout(dropout, name='dropout_1'))
CLSTM.add(LSTM(128, activation='relu', return_sequences=True,
             dropout=dropout, name='lstm_1'))
CLSTM.add(LSTM(128, activation='relu', return_sequences=False,
             dropout=dropout, name='lstm_2'))
CLSTM.add(Dense(units=64, activation='relu', name='fc'))
CLSTM.add(Dropout(dropout, name='dropout_2'))
CLSTM.add(Dense(units=output_dim, activation='softmax', name='softmax'))

# Compile the model 
CLSTM.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [41]:
CLSTM.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv1D)              (None, 2, 256)            443136    
_________________________________________________________________
activation_8 (Activation)    (None, 2, 256)            0         
_________________________________________________________________
batch_normalization (BatchNo (None, 2, 256)            1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 256)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 2, 128)            197120    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
fc (Dense)                   (None, 64)              

In [44]:
#Train the model 
x_train = np.array(mfcc_train)
y_train = np.array(y_train)
CLSTM.fit(x=x_train, y=y_train, epochs=12, batch_size=128)

Epoch 1/12
354/354 [==============================] - 16s 45ms/step - loss: 1.1897 - accuracy: 0.6620
Epoch 2/12
354/354 [==============================] - 14s 40ms/step - loss: 0.8948 - accuracy: 0.7161
Epoch 3/12
354/354 [==============================] - 16s 45ms/step - loss: 0.7662 - accuracy: 0.7499
Epoch 4/12
354/354 [==============================] - 20s 56ms/step - loss: 0.6825 - accuracy: 0.7767
Epoch 5/12
354/354 [==============================] - 16s 45ms/step - loss: 0.6232 - accuracy: 0.7963
Epoch 6/12
354/354 [==============================] - 14s 39ms/step - loss: 0.5878 - accuracy: 0.8053
Epoch 7/12
354/354 [==============================] - 15s 43ms/step - loss: 0.5491 - accuracy: 0.8186
Epoch 8/12
354/354 [==============================] - 13s 37ms/step - loss: 0.5195 - accuracy: 0.8279
Epoch 9/12
354/354 [==============================] - 13s 37ms/step - loss: 0.4973 - accuracy: 0.8364
Epoch 10/12
354/354 [==============================] - 14s 39ms/step - loss: 0.476

In [45]:
# Testing the model 
x_test = np.array(mfcc_test)
y_test = np.array(y_test)
score = CLSTM.evaluate(x_test,y_test)

607/607 [==============================] - 3s 5ms/step - loss: 0.4015 - accuracy: 0.8687
